## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Data download

In [17]:
#Importing gdf usv for series 1 to 7
usv_series1 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series1_pasodelnorte.gpkg")
usv_series2 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series2_pasodelnorte.gpkg")
usv_series3 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series3_pasodelnorte.gpkg")
usv_series4 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series4_pasodelnorte.gpkg")
usv_series5 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series5_pasodelnorte.gpkg")
usv_series6 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series6_pasodelnorte.gpkg")
usv_series7 = gpd.read_file("../../../data/external/temporal_todocker/usodesueloyveg_series7_pasodelnorte.gpkg")

## Data treatment

### Data treatment - Calculate areas

In [30]:
usv_series1_calc = usv_series1.copy()
usv_series1_calc = usv_series1_calc.to_crs("EPSG:6372")
usv_series1_calc['area_has'] = usv_series1_calc['geometry'].area/10000
usv_series1_calc = usv_series1_calc.to_crs("EPSG:4326")
usv_series1_calc.head(2)

,codigo,descripcio,cov_,cov_id,geometry,area_has
0,9.999995e+10,MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL SUB...,4062,4063,"MULTIPOLYGON (((-105.94464 28.54834, -105.9562...",18864.607326
1,2.080707e+10,PASTIZAL NATURAL,9060,9061,"MULTIPOLYGON (((-106.38558 28.93810, -106.3468...",7095.627038


In [31]:
usv_series2_calc = usv_series2.copy()
usv_series2_calc = usv_series2_calc.to_crs("EPSG:6372")
usv_series2_calc['area_has'] = usv_series2_calc['geometry'].area/10000
usv_series2_calc = usv_series2_calc.to_crs("EPSG:4326")
usv_series2_calc.head(2)

,codigo,descripcio,cov_,cov_id,geometry,area_has
0,2.080707e+10,PASTIZAL NATURAL,7032,7033,"MULTIPOLYGON (((-106.38421 28.93738, -106.3443...",6274.419395
1,2.080707e+10,PASTIZAL NATURAL,9982,9983,"MULTIPOLYGON (((-106.18583 28.93708, -106.1708...",2093.775862


In [32]:
usv_series3_calc = usv_series3.copy()
usv_series3_calc = usv_series3_calc.to_crs("EPSG:6372")
usv_series3_calc['area_has'] = usv_series3_calc['geometry'].area/10000
usv_series3_calc = usv_series3_calc.to_crs("EPSG:4326")
usv_series3_calc.head(2)

,codigo,descripcio,cov_,cov_id,geometry,area_has
0,2.080707e+10,PASTIZAL NATURAL,7639,7640,"MULTIPOLYGON (((-106.38421 28.93738, -106.3443...",5166.489379
1,2.080707e+10,PASTIZAL NATURAL,10592,10593,"MULTIPOLYGON (((-106.18583 28.93708, -106.1708...",2093.830552


In [33]:
usv_series4_calc = usv_series4.copy()
usv_series4_calc = usv_series4_calc.to_crs("EPSG:6372")
usv_series4_calc['area_has'] = usv_series4_calc['geometry'].area/10000
usv_series4_calc = usv_series4_calc.to_crs("EPSG:4326")
usv_series4_calc.head(2)

,codigo,descripcio,cov_,cov_id,geometry,area_has
0,2.080707e+10,PASTIZAL NATURAL,7983,7984,"MULTIPOLYGON (((-106.38423 28.93739, -106.3444...",4793.005212
1,2.080707e+10,PASTIZAL NATURAL,11081,11082,"MULTIPOLYGON (((-106.18583 28.93707, -106.1708...",2088.048207


In [34]:
usv_series5_calc = usv_series5.copy()
usv_series5_calc = usv_series5_calc.to_crs("EPSG:6372")
usv_series5_calc['area_has'] = usv_series5_calc['geometry'].area/10000
usv_series5_calc = usv_series5_calc.to_crs("EPSG:4326")
usv_series5_calc.head(2)

,codigo,descripcio,cov_,cov_id,geometry,area_has
0,3.000000e+10,DESPROVISTO DE VEGETACIÃ“N,316,317,"MULTIPOLYGON (((-105.92053 28.64284, -105.9176...",87.968731
1,3.000000e+10,ASENTAMIENTOS HUMANOS,4833,4834,"MULTIPOLYGON (((-106.01526 28.58695, -106.0160...",140.023565


In [35]:
usv_series6_calc = usv_series6.copy()
usv_series6_calc = usv_series6_calc.to_crs("EPSG:6372")
usv_series6_calc['area_has'] = usv_series6_calc['geometry'].area/10000
usv_series6_calc = usv_series6_calc.to_crs("EPSG:4326")
usv_series6_calc.head(2)

,DESCRIPCIO,COV_,COV_ID,codigo,geometry,area_has
0,ÃREA DESPROVISTA DE VEGETACIÃ“N,737,738,3.000000e+10,"MULTIPOLYGON (((-106.13298 28.61224, -106.1331...",15.557905
1,ÃREA DESPROVISTA DE VEGETACIÃ“N,738,739,3.000000e+10,"MULTIPOLYGON (((-105.92053 28.64284, -105.9176...",87.968737


In [36]:
usv_series7_calc = usv_series7.copy()
usv_series7_calc = usv_series7_calc.to_crs("EPSG:6372")
usv_series7_calc['area_has'] = usv_series7_calc['geometry'].area/10000
usv_series7_calc = usv_series7_calc.to_crs("EPSG:4326")
usv_series7_calc.head(2)

,CLAVE,DESCRIPCIO,AREA,usv_svii,cov_,cov_id,geometry,area_has
0,ADV,DESPROVISTO DE VEGETACIÃ“N,287200.262978,3.000000e+10,1217,1218,"MULTIPOLYGON (((-106.12863 28.54289, -106.1276...",28.720026
1,ADV,DESPROVISTO DE VEGETACIÃ“N,160924.300794,3.000000e+10,1220,1221,"MULTIPOLYGON (((-105.91883 28.57913, -105.9192...",16.092430


### Data treatment - Create categories

In [68]:
usv_series1_calc.descripcio.unique()

array(['MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL SUBINERME',
       'PASTIZAL NATURAL', 'BOSQUE DE ENCINO', 'BOSQUE DE PINO-ENCINO',
       'MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL ESPINOSO',
       'AGRICULTURA DE TEMPORAL', 'AGRICULTURA DE RIEGO',
       'VEGETACIÃ“N SECUNDARIA ARBUSTIVA DE BOSQUE DE ENCINO-PINO',
       'PASTIZAL INDUCIDO', 'BOSQUE DE ENCINO-PINO',
       'MATORRAL DESÃ‰RTICO ROSETÃ“FILO CON MATORRAL SUBINERME',
       'PASTIZAL HALÃ“FILO', 'ZONA URBANA',
       'MATORRAL DESÃ‰RTICO MICROFILO Y MATORRAL SUBINERME CON EROSIÃ“N'],
      dtype=object)

In [69]:
series1_cat_tipo = {'Agricultura':['AGRICULTURA DE TEMPORAL','AGRICULTURA DE RIEGO'],
                   'Bosque':['BOSQUE DE ENCINO','BOSQUE DE PINO-ENCINO','BOSQUE DE ENCINO-PINO','VEGETACIÃ“N SECUNDARIA ARBUSTIVA DE BOSQUE DE ENCINO-PINO'],
                   'Matorral':['MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL SUBINERME','MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL ESPINOSO','MATORRAL DESÃ‰RTICO ROSETÃ“FILO CON MATORRAL SUBINERME','MATORRAL DESÃ‰RTICO MICROFILO Y MATORRAL SUBINERME CON EROSIÃ“N'],
                   'Pastizal':['PASTIZAL NATURAL','PASTIZAL INDUCIDO','PASTIZAL HALÃ“FILO'],
                   'Zona urbana':['ZONA URBANA']}

for cat in series1_cat_tipo.keys():
    for tipo in series1_cat_tipo[cat]:
        idx = usv_series1_calc['descripcio'] == tipo
        usv_series1_calc.loc[idx,'category'] = cat
        
        
usv_series1_calc.head()

,codigo,descripcio,cov_,cov_id,geometry,area_has,category
0,9.999995e+10,MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL SUB...,4062,4063,"MULTIPOLYGON (((-105.94464 28.54834, -105.9562...",18864.607326,Matorral
1,2.080707e+10,PASTIZAL NATURAL,9060,9061,"MULTIPOLYGON (((-106.38558 28.93810, -106.3468...",7095.627038,Pastizal
2,2.080707e+10,PASTIZAL NATURAL,14563,14564,"MULTIPOLYGON (((-106.12630 28.96785, -106.1213...",1518.785859,Pastizal
3,2.020101e+10,BOSQUE DE ENCINO,17873,17874,"MULTIPOLYGON (((-106.41491 28.94564, -106.4012...",4563.881361,Bosque
4,2.010501e+10,BOSQUE DE PINO-ENCINO,17983,17984,"MULTIPOLYGON (((-106.44965 28.76188, -106.4585...",8666.578931,Bosque


In [70]:
series1_catareas = usv_series1_calc.groupby('category').agg({'area_has':np.sum})
series1_catareas

,area_has
category,
Agricultura,21533.253292
Bosque,56506.187201
Matorral,145506.748810
Pastizal,34407.996160
Zona urbana,3217.183308


## Data analysis

### Data analysis - Areas by 'descripcio'

In [37]:
series1_areas = usv_series1_calc.groupby('descripcio').agg({'area_has':np.sum})
series1_areas

,area_has
descripcio,
AGRICULTURA DE RIEGO,9099.442284
AGRICULTURA DE TEMPORAL,12433.811008
BOSQUE DE ENCINO,44692.490456
BOSQUE DE ENCINO-PINO,2492.327038
BOSQUE DE PINO-ENCINO,9273.069635
MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL ESPINOSO,56506.274946
MATORRAL DESÃ‰RTICO MICROFILO CON MATORRAL SUBINERME,85328.602253
MATORRAL DESÃ‰RTICO MICROFILO Y MATORRAL SUBINERME CON EROSIÃ“N,1673.991066
MATORRAL DESÃ‰RTICO ROSETÃ“FILO CON MATORRAL SUBINERME,1997.880546


In [38]:
series2_areas = usv_series2_calc.groupby('descripcio').agg({'area_has':np.sum})
series2_areas

,area_has
descripcio,
AGRICULTURA DE RIEGO ANUAL,5192.262649
AGRICULTURA DE RIEGO ANUAL Y SEMIPERMANENTE,4064.099190
AGRICULTURA DE RIEGO SEMIPERMANENTE,1898.684888
AGRICULTURA DE TEMPORAL ANUAL,16796.755377
BOSQUE DE ENCINO,46220.462705
BOSQUE DE ENCINO-PINO,1919.097233
BOSQUE DE PINO-ENCINO,9157.643063
CUERPO DE AGUA,300.098888
MATORRAL DESÃ‰RTICO MICRÃ“FILO,86911.778718


In [39]:
series3_areas = usv_series3_calc.groupby('descripcio').agg({'area_has':np.sum})
series3_areas

,area_has
descripcio,
AGRICULTURA DE RIEGO ANUAL,4648.885036
AGRICULTURA DE RIEGO ANUAL Y SEMIPERMANENTE,7371.568730
AGRICULTURA DE TEMPORAL ANUAL,19325.539257
ASENTAMIENTOS HUMANOS,3364.022529
BOSQUE DE ENCINO,45810.973486
BOSQUE DE ENCINO-PINO,1919.039864
BOSQUE DE PINO-ENCINO,9157.661804
CUERPO DE AGUA,300.113159
MATORRAL DESÃ‰RTICO MICRÃ“FILO,78616.765962


In [40]:
series4_areas = usv_series4_calc.groupby('descripcio').agg({'area_has':np.sum})
series4_areas

,area_has
descripcio,
AGRICULTURA DE RIEGO ANUAL,4848.416813
AGRICULTURA DE RIEGO ANUAL Y SEMIPERMANENTE,7216.739415
AGRICULTURA DE TEMPORAL ANUAL,22006.085253
ASENTAMIENTOS HUMANOS,9560.548408
BOSQUE DE ENCINO,45808.071443
BOSQUE DE ENCINO-PINO,1916.697074
BOSQUE DE PINO-ENCINO,9151.522223
CUERPO DE AGUA,300.101618
MATORRAL DESÃ‰RTICO MICRÃ“FILO,71733.912063


In [41]:
series5_areas = usv_series5_calc.groupby('descripcio').agg({'area_has':np.sum})
series5_areas

,area_has
descripcio,
AGRICULTURA DE RIEGO ANUAL,4378.874663
AGRICULTURA DE RIEGO ANUAL Y SEMIPERMANENTE,6959.955210
AGRICULTURA DE RIEGO PERMANENTE,44.330874
AGRICULTURA DE TEMPORAL ANUAL,20491.029731
ASENTAMIENTOS HUMANOS,11778.461406
BOSQUE DE ENCINO,45997.466101
BOSQUE DE ENCINO-PINO,1917.431074
BOSQUE DE PINO-ENCINO,9154.629951
CUERPO DE AGUA,300.092678


In [43]:
series6_areas = usv_series6_calc.groupby('DESCRIPCIO').agg({'area_has':np.sum})
series6_areas

,area_has
DESCRIPCIO,
AGRICULTURA DE RIEGO ANUAL,4454.195671
AGRICULTURA DE RIEGO ANUAL Y SEMIPERMANENTE,6966.458544
AGRICULTURA DE RIEGO PERMANENTE,44.330879
AGRICULTURA DE TEMPORAL ANUAL,20097.326753
AGUA,300.092685
BOSQUE DE ENCINO,45873.155041
BOSQUE DE ENCINO-PINO,1917.431100
BOSQUE DE GALERÃA,69.181953
BOSQUE DE PINO-ENCINO,9278.933427


In [44]:
series7_areas = usv_series7_calc.groupby('DESCRIPCIO').agg({'area_has':np.sum})
series7_areas

,area_has
DESCRIPCIO,
AGRICULTURA DE RIEGO ANUAL,3743.765956
AGRICULTURA DE RIEGO ANUAL Y SEMIPERMANENTE,5552.523934
AGRICULTURA DE RIEGO PERMANENTE,654.150333
AGRICULTURA DE TEMPORAL ANUAL,15816.150030
ASENTAMIENTOS HUMANOS,28820.769148
BOSQUE DE ENCINO,45280.555388
BOSQUE DE ENCINO-PINO,1917.431097
BOSQUE DE GALERÃA,101.091238
BOSQUE DE PINO-ENCINO,9889.424864
